In [1]:
import numpy as np
import pandas as pd
import time
import pickle

In [2]:
from counterfactuals.global_cfs_utils.datasets_split import dataset_loader_split

In [3]:
name = 'german_credit'  # from compas, german_credit, adult_income, heloc
dropped_features = []#UCIDatasets().continuous_features[dataset]
dataset = dataset_loader_split(name, dropped_features=dropped_features, n_bins=None)

/home/lukasz/genwro/counterfactuals/counterfactuals/global_cfs_utils/datasets.py:150: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_address, header = None, delim_whitespace = True)


In [4]:
x_train, y_train, x_test, y_test, x_means, x_std = dataset.get_split(normalise=False, shuffle=False,
                                                                     return_mean_std=True)
prop1s = round(np.average(y_train)*100, 2)
print("Proportion of 1s in Training Data:" + " {}%".format(prop1s))

Proportion of 1s in Training Data: 70.0%


In [5]:
X = pd.DataFrame(x_train)
X.columns = dataset.features[:-1]
X_test = pd.DataFrame(x_test)
X_test.columns = dataset.features[:-1]
print("Dataset:" + " {}\n".format(name.replace('_', ' ').title()))
X

Dataset: German Credit



,Existing-Account-Status = A11,Existing-Account-Status = A12,Existing-Account-Status = A13,Existing-Account-Status = A14,Month-Duration,Credit-History = A30,Credit-History = A31,Credit-History = A32,Credit-History = A33,Credit-History = A34,...,Job = A171,Job = A172,Job = A173,Job = A174,Num-People = 1,Num-People = 2,Telephone = A191,Telephone = A192,Foreign-Worker = A201,Foreign-Worker = A202
0,1,0,0,0,12,0,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0
1,0,0,0,1,36,0,0,1,0,0,...,0,0,1,0,1,0,1,0,1,0
2,0,0,0,1,15,0,0,1,0,0,...,0,0,1,0,1,0,1,0,1,0
3,0,1,0,0,15,0,1,0,0,0,...,0,0,0,1,0,1,0,1,1,0
4,1,0,0,0,30,0,0,0,0,1,...,0,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,1,0,9,0,0,1,0,0,...,0,1,0,0,1,0,1,0,1,0
796,1,0,0,0,6,0,0,1,0,0,...,0,0,1,0,1,0,0,1,1,0
797,0,0,0,1,36,0,0,0,1,0,...,0,0,1,0,1,0,1,0,1,0
798,1,0,0,0,12,0,0,1,0,0,...,0,0,0,1,1,0,0,1,1,0


# Select Model

In [6]:
from counterfactuals.global_cfs_utils.models import performance

In [7]:
B_name = 'lr'  # or 'lr' or 'xgb'
if B_name == 'dnn':
    from counterfactuals.global_cfs_utils.models import dnn_normalisers as normalisers
elif B_name == 'lr':
    from counterfactuals.global_cfs_utils.models import lr_normalisers as normalisers
else:  # no xgb normalisation
    normalisers = {name: False}
with open('../counterfactuals/global_cfs_utils/models/{}_{}.pkl'.format(name, B_name), 'rb') as f:
    B = pickle.load(f)
normalise = [x_means, x_stds] if normalisers[name] else None

/home/lukasz/miniconda3/envs/cf/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
print("Dataset:" + " {}".format(name.replace('_', ' ').title()))
print("Model:" + " {}\n".format(B_name.replace('_', ' ').upper()))
performance(B, x_train, x_test, y_train, y_test, normalise=normalise)

Dataset: German Credit
Model: LR

Train Accuracy: 76.88%
Test Accuracy: 83.5%
Proportion of 1s Predicted (Train): 76.12%
Proportion of 1s Predicted (Test): 76.5%


# AReS

In [9]:
from counterfactuals.global_cfs_utils.ares import AReS

In [10]:
ares = AReS(model=B, dataset=dataset, X=X, dropped_features=[],
            n_bins=10, ordinal_features=[], normalise=normalisers[name],
            constraints=[20,7,10])

In [11]:
# Compute candidate sets of conditions (SD and RL)
ares.generate_itemsets(apriori_threshold=0.2, max_width=None, # defaults to e2-1
                       affected_subgroup=None, save_copy=True)

/home/lukasz/miniconda3/envs/cf/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Computing Candidate Sets of Conjunctions of Predicates SD and RL
Processing 684 combinations | Sampling itemset size 65
SD and RL Computed with Lengths 1265 and 1265
Saving Copies of SD and RL as SD_copy and RL_copy


In [12]:
ares.generate_groundset(max_width=None, RL_reduction=True,
                        then_generation=None, save_copy=False)

Computing Ground Set of Triples V
Reducing RL
RL Reduced from Size 1265 to 389


100%|██████████| 1265/1265 [00:07<00:00, 168.32it/s]

Ground Set Computed with Length 220454


In [13]:
# custom objective function with [accuracy, average cost]
# set lams = [_, _, _, _] for original AReS
lams = [1, 0]

# save_mode: 1 and 2 correspond to r and r' in the Fast AReS paper
# save_mode: 1 and 2 will overwrite the ground set, 0 will not
# use save_mode=0 alongside plot_accuracy=True first
# to get an idea of how many triples you need to evaluate
ares.evaluate_groundset(lams=lams, r=3000, save_mode=1,
                        disable_tqdm=False, plot_accuracy=False)

0it [00:00, ?it/s]/home/lukasz/genwro/counterfactuals/counterfactuals/global_cfs_utils/ares.py:900: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[17.2 11.2 17.2 17.2 17.2 17.2 17.2]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  triple_cfx[triple_cover] = cfx
/home/lukasz/genwro/counterfactuals/counterfactuals/global_cfs_utils/ares.py:900: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[18.8 21.8 18.8 18.8 18.8 18.8 18.8 18.8 18.8 18.8]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  triple_cfx[triple_cover] = cfx
/home/lukasz/genwro/counterfactuals/counterfactuals/global_cfs_utils/ares.py:900: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2514.4 3250.4 315

In [14]:
X_cfs = ares.V.cfx_matrix[-1]

In [15]:
from counterfactuals.metrics.metrics import continuous_distance, categorical_distance, distance_l2_jaccard, distance_mad_hamming, sparsity, perc_valid_cf

In [18]:
X_cf = X_cfs
X = ares.X_aff_original.values
X_test = X_test.values

In [19]:
categorical_features = range(X_cf.shape[1])
continuous_features = range(X_cf.shape[1])

model_returned = np.ones(X_cf.shape[0]).astype(bool)
model_returned_smth = np.sum(model_returned) / len(model_returned)

ys_cfs_disc_pred = np.array(ares.model.predict(X_cfs))

valid_cf_disc_metric = perc_valid_cf(np.zeros_like(ys_cfs_disc_pred), y_cf=ys_cfs_disc_pred)


hamming_distance_metric = categorical_distance(
    X=X, X_cf=X_cf, categorical_features=categorical_features, metric="hamming", agg="mean"
)
jaccard_distance_metric = categorical_distance(
    X=X, X_cf=X_cf, categorical_features=categorical_features, metric="jaccard", agg="mean"
)
manhattan_distance_metric = continuous_distance(
    X=X, X_cf=X_cf, continuous_features=continuous_features, metric="cityblock", X_all=X_test
)
euclidean_distance_metric = continuous_distance(
    X=X, X_cf=X_cf, continuous_features=continuous_features, metric="euclidean", X_all=X_test
)
mad_distance_metric = continuous_distance(
    X=X, X_cf=X_cf, continuous_features=continuous_features, metric="mad", X_all=X_test
)
l2_jaccard_distance_metric = distance_l2_jaccard(
    X=X,
    X_cf=X_cf,
    continuous_features=continuous_features,
    categorical_features=categorical_features,
)
mad_hamming_distance_metric = distance_mad_hamming(
    X=X,
    X_cf=X_cf,
    continuous_features=continuous_features,
    categorical_features=categorical_features,
    X_all=X_test,
    agg="mean",
)
sparsity_metric = sparsity(X, X_cf)

metrics = {
    "model_returned_smth": model_returned_smth,
    "valid_cf_disc": valid_cf_disc_metric,
    "dissimilarity_proximity_categorical_hamming": hamming_distance_metric,
    "dissimilarity_proximity_categorical_jaccard": jaccard_distance_metric,
    "dissimilarity_proximity_continuous_manhatan": manhattan_distance_metric,
    "dissimilarity_proximity_continuous_euclidean": euclidean_distance_metric,
    "dissimilarity_proximity_continuous_mad": mad_distance_metric,
    "distance_l2_jaccard": l2_jaccard_distance_metric,
    "distance_mad_hamming": mad_hamming_distance_metric,
    "sparsity": sparsity_metric,
}